In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *
import warnings
warnings.simplefilter('ignore')
from xgboost import XGBRegressor

In [3]:
df=pd.read_excel('Data_Train.xlsx')

In [4]:
df=df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10682 non-null  object
 1   Date_of_Journey  10682 non-null  object
 2   Source           10682 non-null  object
 3   Destination      10682 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10682 non-null  object
 6   Arrival_Time     10682 non-null  object
 7   Duration         10682 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10682 non-null  object
 10  Price            10682 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 1001.4+ KB


In [6]:
df['Airline'].value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1751
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [7]:
df['Day']=df['Date_of_Journey'].apply(lambda x:x.split('/')[0])

In [9]:
df['Month']=df['Date_of_Journey'].apply(lambda x:x.split('/')[1])
df['Year']=df['Date_of_Journey'].apply(lambda x:x.split('/')[2])

In [10]:
df=df.drop('Date_of_Journey',axis=1)


In [11]:
df['Day']=df['Day'].apply(lambda x:int(x))
df['Month']=df['Month'].apply(lambda x:int(x))
df['Year']=df['Year'].apply(lambda x:int(x))

In [12]:
df=df.drop(['Arrival_Time','Dep_Time'],axis=1)

In [13]:
df['Hours']=df['Duration'].apply(lambda x:x.split()[0])
df['Minutes']=df['Duration'].apply(lambda x:x.split()[1] if len(x.split())>1 else 0)

In [14]:
df['Hours']=df['Hours'].apply(lambda x:x[0])
df['Minutes']=df['Minutes'].apply(lambda x:x[:x.index('m')]if x!=0 else 0)

In [15]:
df['Hours']=df['Hours'].apply(lambda x:int(x))
df['Minutes']=df['Minutes'].apply(lambda x:int(x))

In [16]:
df=df.drop(['Duration','Route'],axis=1)

In [17]:
df['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [18]:
df['Additional_Info']=np.where(df['Additional_Info']=='No Info','No info',df['Additional_Info'])

In [19]:
df['Additional_Info'].value_counts()

No info                         8347
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
2 Long layover                     1
1 Short layover                    1
Red-eye flight                     1
Name: Additional_Info, dtype: int64

In [20]:
df['Total_duration']=(df['Hours']*60)+df['Minutes']

In [21]:
df=df.drop(['Hours','Minutes'],axis=1)

In [22]:
df['Total_Stops']=np.where(df['Total_Stops']!='non-stop','stop',df['Total_Stops'])

In [23]:
X=df.drop('Price',axis=1)
y=df.Price
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
cat_cols=[i for i in x_train.columns if x_train[i].dtypes=='O']
num_cols=[i for i in x_train.columns if x_train[i].dtypes!='O']

In [24]:
label=LabelEncoder()
for i in cat_cols:
  x_train[i]=label.fit_transform(x_train[i])
  print(label.classes_,label.transform(label.classes_))
  x_test[i]=label.transform(x_test[i])

['Air Asia' 'Air India' 'GoAir' 'IndiGo' 'Jet Airways'
 'Jet Airways Business' 'Multiple carriers'
 'Multiple carriers Premium economy' 'SpiceJet' 'Trujet' 'Vistara'
 'Vistara Premium economy'] [ 0  1  2  3  4  5  6  7  8  9 10 11]
['Banglore' 'Chennai' 'Delhi' 'Kolkata' 'Mumbai'] [0 1 2 3 4]
['Banglore' 'Cochin' 'Delhi' 'Hyderabad' 'Kolkata' 'New Delhi'] [0 1 2 3 4 5]
['non-stop' 'stop'] [0 1]
['1 Long layover' '1 Short layover' '2 Long layover' 'Business class'
 'Change airports' 'In-flight meal not included'
 'No check-in baggage included' 'No info' 'Red-eye flight'] [0 1 2 3 4 5 6 7 8]


In [25]:
pp=Pipeline([
             ('step2',StandardScaler()),
            #  ('Step3',PolynomialFeatures(degree=3)),
            #  ('model',LinearRegression())
            ('model',RandomForestRegressor(n_estimators=300,min_samples_split=18,max_depth=15))
])
y_pred=pp.fit(x_train,y_train).predict(x_test)
mean_squared_error(y_test,y_pred)

4930352.630512268

In [26]:
r2_score(y_test,y_pred)

0.7898684795581863

In [27]:
print(y_pred)

[14175.04241856  6329.88120531  3651.29897986 ...  8814.20020355
  9762.75106156  4020.30674405]


In [29]:
prediction = pd.DataFrame(y_pred, columns=['predictions']).to_excel('Test_set.xlsx')